In [1]:
import json
import pymysql

# 读取配置文件
with open('weibo_config.json', 'r') as f:
    config = json.load(f)

`user`表属性：

- `id`：（唯一）用户ID
- `screen_name`：用户昵称
- `gender`：性别
- `statuses_count`：（总）微博数

`weibo`表属性：

- `id`：（唯一）微博ID
- `user_id`：用户ID
- `text`：微博正文
- `created_at`：微博发布时间
- `attitudes_count`：微博点赞数
- `comments_count`：微博评论数
- `reposts_count`：微博转发数


In [2]:
user_id_ip_file = r"collecting\user_id_ip.txt"

with open(user_id_ip_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    id_ip_dict = {line.split(',')[0]: line.strip().split(',')[1] for line in lines}

In [3]:

def execute_sql(sql, params=None):
    try:
        with pymysql.connect(**config['database']) as conn:
            with conn.cursor() as cursor:
                cursor.execute(sql, params)
                result = cursor.fetchall()
                return result
    except Exception as e:
        print(f"执行SQL语句时出现错误：{e}")
        raise


In [8]:
def execute_many_sql(sql, param_list):
    try:
        with pymysql.connect(**config['database']) as conn:
            with conn.cursor() as cursor:
                cursor.executemany(sql, param_list)
            conn.commit()
    except Exception as e:
        print(f"批量执行SQL时出错：{e}")
        raise

sql = "UPDATE `user` SET IP = %s WHERE id = %s"
param_list = [(ip, uid) for uid, ip in id_ip_dict.items()]
execute_many_sql(sql, param_list)

In [ ]:

user_attributes = ("user_id", "screen_name", "sex", "statuses_count")
weibo_attributes = ("user_id", "weibo_id", "text", "created_at", "attitudes_count", "comments_count", "reposts_count")





select_query = """
SELECT id, screen_name, gender, statuses_count
FROM `user`
WHERE to_be_cleaned != 1
LIMIT 100;
"""

users = execute_sql(select_query)

user_dict = {}
for u in users:
    user = dict(zip(user_attributes, u))
    user["IP"] = id_ip_dict.get(user["user_id"], "未知")
    user_dict[user["user_id"]] = user


In [4]:
id_list = [user['user_id'] for user in user_dict.values()]

placeholders = ', '.join(['%s'] * len(id_list))

select_query = f"""
SELECT 
    user_id, 
    id, 
    text, 
    DATE_FORMAT(created_at, '%%Y-%%m-%%dT%%H:%%i:%%s'), 
    attitudes_count, 
    comments_count, 
    reposts_count
FROM `weibo`
WHERE user_id IN ({placeholders})
"""

weibos_of_users = execute_sql(select_query, id_list)


In [6]:

for user_id, user_info in user_dict.items():
    weibos = [dict(zip(weibo_attributes[1:], weibo[1:])) 
              for weibo in weibos_of_users
              if weibo[0] == user_id]
    weibos.sort(key=lambda x: x["created_at"])
    user_info["weibo"] = weibos

In [5]:
import random

user_dict["1000129923"]


{'user_id': '1000129923',
 'screen_name': '重铸无限城荣光',
 'sex': 'f',
 'statuses_count': 7991,
 'IP': '湖南'}

In [38]:
with open('test100.json', 'w', encoding='utf-8') as f:
    json.dump(user_dict, f, ensure_ascii=False, indent=4)